In [17]:
#v3.classification
#28/11/2018

dataname="pca_detect_dense_rpF"

patch_size=224 #size of the tiles to extract and save in the database, must be >= to training size
stride_size=224 #distance to skip between patches, 1 indicated pixel wise extraction, patch_size would result in non-overlapping tiles
mirror_pad_size=0 # number of pixels to pad *after* resize to image with by mirroring (edge's of patches tend not to be analyzed well, so padding allows them to appear more centered in the patch)
test_set_size=.2 # what percentage of the dataset should be used as a held out validation/testing set

desired_mask_mpp = 16.1 # what we would like to use for finding valid regions of the mask
model_mpp = 2 # MPP of patches to be fed into model
max_patches_per_image = 5000

biopsy = True
use_hsv = False
remove_white = False

positive_class = 'green'
if biopsy:
    negative_class = 'blue'
else:       
    negative_class = 'yellow' # prev:yellow


class_names=["benign", "tumor"]#what classes we expect to have in the data, here we have only 2 classes but we could add additional classes

#-----Note---
#One should likely make sure that  (nrow+mirror_pad_size) mod patch_size == 0, where nrow is the number of rows after resizing
#so that no pixels are lost (any remainer is ignored)


In [18]:
import torch
import tables

import os,sys
import glob

import PIL
import numpy as np

import cv2
import matplotlib.pyplot as plt

from sklearn import model_selection
import sklearn.feature_extraction.image
import random

from WSI_handling import wsi

from pathlib import Path
from scipy.ndimage import binary_opening
seed = random.randrange(sys.maxsize) #get a random seed so that we can reproducibly do the cross validation setup
random.seed(seed) # set the seed
print(f"random seed (note down for reproducibility): {seed}") # 3606024316055270146

random seed (note down for reproducibility): 5536355547312335575


In [19]:
def check_for_classes(img_fname,positive_class,negative_class):    
    if(os.path.exists(Path(img_fname).with_suffix('.xml')) or os.path.exists(Path(img_fname).with_suffix('.json'))):
        w = wsi(None, Path(img_fname).with_suffix('.xml'))
        points, map_idx, _, _ = w.get_points(colors_to_use=[positive_class,negative_class],custom_colors = [])
        return len(np.unique(map_idx))>1
    else:
        return False
def check_for_classes_biop(img_fname,positive_class,negative_class):    
    if(os.path.exists(Path(img_fname).with_suffix('.xml')) or os.path.exists(Path(img_fname).with_suffix('.json'))):
        w = wsi(None, Path(img_fname).with_suffix('.xml'))
        points, map_idx, _, _ = w.get_points(colors_to_use=[positive_class,negative_class],custom_colors = [])
        return len(np.unique(map_idx))>1 # used to be 0, changed it to try to improve class imbalance




In [20]:
img_dtype = tables.UInt8Atom()  # dtype in which the images will be saved, this indicates that images will be saved as unsigned int 8 bit, i.e., [0,255]
filenameAtom = tables.StringAtom(itemsize=255) #create an atom to store the filename of the image, just incase we need it later, 

In [21]:
seed = 7331385269455319306 # 4622693214520544803
# random.seed(seed) # set the seed
if biopsy:
    files=glob.glob(r'../../../../data/UPenn_Prostate_Histology/UPenn_Prostate_Biopsy/*.svs') # bps
    files = [fname for fname in files if check_for_classes_biop(fname,positive_class,negative_class)]
else:
    files=glob.glob(r'../../../../data/UPenn_Prostate_Histology/Progressor_nonProgressorProstate/histologyImages/UPenn/*.svs') #rps
    files = [fname for fname in files if check_for_classes(fname,positive_class,negative_class)]
    
#create training and validation stages and split the files appropriately between them
print(len(files))
phases={}
phases["train"],phases["val"]=next(iter(model_selection.ShuffleSplit(n_splits=1,test_size=test_set_size, random_state = 5).split(files)))
print(phases['val'])

24
[20  2 17 18 19]


In [22]:
startFile = {'train': 0, 'val':0}
totalsDict = {'train': np.array([0,0]), 'val': np.array([0,0])}
storage={} #holder for future pytables

block_shape=np.array((patch_size,patch_size,3)) #block shape specifies what we'll be saving into the pytable array, here we assume that masks are 1d and images are 3d

filters=tables.Filters(complevel=6, complib='zlib') #we can also specify filters, such as compression, to improve storage speed


# for phase in phases.keys(): #now for each of the phases, we'll loop through the files
for phase in ['train', 'val']:
    print(phase)
    #totals=np.zeros(len(class_names)) # we can to keep counts of all the classes in for in particular training, since we 
    totals = totalsDict[phase]
    hdf5_file = tables.open_file(f"./{dataname}_{phase}.pytable", mode='a') #open the respective pytable
    storage["filenames"] = hdf5_file.create_earray(hdf5_file.root, 'filenames', filenameAtom, (0,)) #create the array for storage
    storage["imgs"] = hdf5_file.create_earray(hdf5_file.root, "imgs", img_dtype,  shape=np.append([0],block_shape), chunkshape=np.append([1],block_shape),filters=filters)
    storage["labels"]= hdf5_file.create_earray(hdf5_file.root, "labels", img_dtype,  shape=[0], chunkshape=[1],filters=filters)
    npixels = hdf5_file.create_carray(hdf5_file.root, 'classsizes', tables.Atom.from_dtype(totals.dtype), totals.shape)

    for i, filei in enumerate(phases[phase]): #now for each of the files
        fname=files[filei] 
        print(fname)

        wsi_img=wsi(fname,Path(fname).with_suffix('.xml'))    

        #stride_size_converted = wsi_img.get_coord_at_mpp(stride_size,output_mpp=desired_mask_mpp,input_mpp=wsi_img["mpp"])
        stride_size_converted = wsi_img.get_coord_at_mpp(stride_size,output_mpp=desired_mask_mpp,input_mpp=model_mpp) 

        mask_small = wsi_img.mask_out_annotation(desired_mpp=desired_mask_mpp,colors_to_use=(negative_class,positive_class))            

        if biopsy:
            small_img = wsi_img.get_wsi(desired_mpp=desired_mask_mpp)
            tissue_mask = (np.mean(small_img,axis=2) < 220) * 1 # anything that is not white
            tissue_mask = binary_opening(tissue_mask, np.ones((5,5))) * 1
            tissue_mask[mask_small > 0] = 2 
            mask_small = tissue_mask


        mask_small = mask_small[list(range(0,np.shape(mask_small)[0],stride_size_converted)),:]                    
        mask_small = mask_small[:,list(range(0,np.shape(mask_small)[1],stride_size_converted))]

        [rs,cs]=(mask_small>0).nonzero()
        
        print(len(rs))
        edge = .05
        r_keep = [i and j for i,j in zip((rs > (edge * np.shape(mask_small)[0])), rs < (((1-edge) * np.shape(mask_small)[0])))]
        c_keep = [i and j for i,j in zip((cs > (edge * np.shape(mask_small)[1])), cs < (((1-edge) * np.shape(mask_small)[1])))]
        keep_s = [i and j for i,j in zip(r_keep, c_keep)]

        rs = rs[keep_s]
        cs = cs[keep_s]
        
        print(len(rs))
        rscs_labels = [mask_small[rscs[0],rscs[1]] for rscs in zip(rs,cs)]

        rs = [wsi_img.get_coord_at_mpp(r*stride_size_converted,wsi_img["mpps"][0],desired_mask_mpp) for r in rs]
        cs = [wsi_img.get_coord_at_mpp(c*stride_size_converted,wsi_img["mpps"][0],desired_mask_mpp) for c in cs]

        goods = np.ones(np.shape(rs)[0])
        for k in range(0,np.shape(rs)[0]):

            te_tile = wsi_img.get_tile(coords=(cs[k],rs[k]),wh=(3,3),desired_mpp=desired_mask_mpp)

            # check if tile has too high a fraction of white pixels, will do a full check later
            if(np.all(te_tile>220)):
                goods[k] = False             

        rs = [r for idx,r in enumerate(rs) if goods[idx]]
        cs = [c for idx,c in enumerate(cs) if goods[idx]]
        rscs_labels = [rscs_label for idx,rscs_label in enumerate(rscs_labels) if goods[idx]]

        rscs = [(cs[k],rs[k]) for k in range(0,np.size(rs)-1)]

        n_images = min(max_patches_per_image,len(rscs))
        rscs_labels = [rscs_labels[k] for k in np.linspace(0,len(rscs)-1,n_images,dtype='int')]
        rscs = [rscs[k] for k in np.linspace(0,len(rscs)-1,n_images,dtype='int')]      
        for k,rcpair in enumerate(rscs):
            img = wsi_img.get_tile(desired_mpp=model_mpp,coords=(rcpair[0],rcpair[1]),wh=(patch_size,patch_size))
            annot = (wsi_img.mask_out_tile(desired_mpp=model_mpp, coords=(rcpair[0],rcpair[1]), wh=(patch_size,patch_size)) > 0) * 1
            if((np.sum(np.mean(img,axis=2)>220)/np.size(img[:,:,1]))<0.70): # 0.70
#                 if np.var(np.mean(img, axis=2)) < 200:
                if np.sum(annot)/(np.square(np.shape(annot)[0])) > .2:
                    classid = 1
                else:
                    classid = 0
                # classid = (rscs_labels[k] - 1) # biopsies
                white_mask = np.mean(img,axis=2)>220
                if remove_white:
                    for i in range(3):
                        temp = img[:,:,i]
                        temp[white_mask] = 0
                        img[:,:,i] = temp
                if use_hsv:
                    hsv = np.uint8(cv2.cvtColor(img, cv2.COLOR_RGB2HSV))
                    storage["imgs"].append(np.expand_dims(hsv,0))
                else:
                    storage["imgs"].append(np.expand_dims(img,0))
                storage["labels"].append(np.expand_dims(classid,0)) #add the filename to the storage array
                storage["filenames"].append(np.expand_dims(fname,0)) #add the filename to the storage array
                totals[classid]+=1
        print(totals)
    #lastely, we should store the number of pixels
    npixels[:]=totals
    hdf5_file.close()

train
../../../../data/UPenn_Prostate_Histology/UPenn_Prostate_Biopsy/51755.svs
66
60
[35  6]
../../../../data/UPenn_Prostate_Histology/UPenn_Prostate_Biopsy/52619.svs
76
74
[69 25]
../../../../data/UPenn_Prostate_Histology/UPenn_Prostate_Biopsy/52610.svs
130
107
[125  40]
../../../../data/UPenn_Prostate_Histology/UPenn_Prostate_Biopsy/51554.svs
91
81
[157  57]
../../../../data/UPenn_Prostate_Histology/UPenn_Prostate_Biopsy/51455.svs
49
48
[189  61]
../../../../data/UPenn_Prostate_Histology/UPenn_Prostate_Biopsy/51536.svs
98
97
[233  85]
../../../../data/UPenn_Prostate_Histology/UPenn_Prostate_Biopsy/51461.svs
105
101
[298  94]
../../../../data/UPenn_Prostate_Histology/UPenn_Prostate_Biopsy/51558.svs
119
118
[348 138]
../../../../data/UPenn_Prostate_Histology/UPenn_Prostate_Biopsy/51539.svs
67
66
[386 151]
../../../../data/UPenn_Prostate_Histology/UPenn_Prostate_Biopsy/51459.svs
51
46
[421 156]
../../../../data/UPenn_Prostate_Histology/UPenn_Prostate_Biopsy/51572.svs
40
40
[440 165]
..

In [23]:
#useful reference
#http://machinelearninguru.com/deep_learning/data_preparation/hdf5/hdf5.html

In [24]:
print(np.unique(mask_small))

[0 1 2]
